## Login to Hugging Face

In [1]:
from dotenv import load_dotenv
import os
from huggingface_hub import login

load_dotenv()
token = os.getenv("HUGGINGFACE_TOKEN")
login(
    token=token, # ADD YOUR TOKEN HERE
    add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/pathfinder/.cache/huggingface/token
Login successful


## Imports

In [2]:
# pytorch
import torch

# huggingface
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)

## Device

In [3]:
device = (
    "cuda:0" if torch.cuda.is_available() else # Nvidia GPU
    "mps" if torch.backends.mps.is_available() else # Apple Silicon GPU
    "cpu"
)
print(f"Device = {device}")

Device = cuda:0


## Hyperparameters

In [4]:
# Tokenizer arguments
max_length = 1024

# model arguments
max_new_tokens=500

# mixed precision
dtype = torch.bfloat16

# quantization configuration
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=dtype,
    bnb_4bit_quant_type="nf4"
)

## Model

In [5]:
# Model List

# gemma variants
# "google/gemma-7b-it" // downloaded
# "PathFinderKR/waktaverse-gemma-ko-7b-it"
# "beomi/gemma-ko-7b"

# llama2 variants
# "meta-llama/Llama-2-7b-chat-hf" // downloaded
# "PathFinderKR/waktaverse-Llama-2-ko-7b-it"
# "beomi/KoAlpaca-Polyglot-5.8B" // downloaded
# "beomi/open-llama-2-ko-7b"

# solar variants
# "chihoonlee10/T3Q-ko-solar-dpo-v4.0"

In [6]:
model_id = "PathFinderKR/waktaverse-gemma-ko-7b-it"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map=device,
    attn_implementation="flash_attention_2",
    torch_dtype=dtype,
    quantization_config=quantization_config
)

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/522 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

## Running the model on a single GPU

In [8]:
# Chat Template
def generate_response(prompt):
    messages = [{"role": "user", "content": prompt}]
    chat = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    input_ids = tokenizer.encode(chat, add_special_tokens=False, return_tensors="pt")
    outputs = model.generate(input_ids=input_ids.to(model.device), max_new_tokens=max_new_tokens)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [9]:
#prompt = "Write me a poem about Machine Learning."
#prompt = "머신러닝에 대한 시를 써주세요."

In [10]:
prompt = "코루틴에 대해 설명해줘"

In [11]:
response = generate_response(prompt)
print(response)

user
코루틴에 대해 설명해줘
model
**코루틴**

코루틴은 비동기 프로그래밍 개념의 하나입니다. 코루틴은 스스로가 동시에 실행되는 것 같지만, 실제는 스택을 사용하지 않고 대기 상태로 넘어갑니다. 코루틴은 다른 코루틴이 완료되거나 이벤트가 발생할 때까지 대기합니다.

**주요 특징:**

* **비동기:** 코루틴은 동시에 실행되지만, 대기 상태로 넘어갑니다.
* **스택 없이:** 코루틴은 스택을 사용하지 않고 대기 상태로 넘어갑니다.
* **순차적:** 코루틴은 순차적으로 실행됩니다.
* **상태:** 코루틴은 상태를 유지하지 않고 대기 상태로 넘어갑니다.
* **콜백:** 코루틴이 완료되거나 이벤트가 발생하면 콜백 함수가 호출됩니다.
* **정합:** 코루틴은 다른 코루틴이 동시에 실행되는 것 같지만, 실제는 순차적으로 실행됩니다.

**장점:**

* **순차적:** 코루틴은 순차적으로 실행되므로 코드를 더 간단하게 작성할 수 있습니다.
* **비동기:** 코루틴은 비동기 작업을 쉽게 처리할 수 있습니다.
* **상태 없이:** 코루틴은 상태를 유지하지 않고 대기 상태로 넘어갑니다.
* **콜백:** 코루틴이 완료되거나 이벤트가 발생하면 콜백 함수가 호출됩니다.

**단점:**

* **순차적 제한:** 코루틴은 순차적으로 실행되므로, 동시에 수행하는 작업에는 제한이 있습니다.
* **대기:** 코루틴은 대기 상태로 넘어갑니다. 따라서 모든 코루틴이 완료되거나 이벤트가 발생하기 전까지 대기해야 합니다.
* **사고:** 코루틴을 사용하는 데는 숙련이 필요합니다.

**예제:**

```python
def hello(name):
  print("Hello, " + name)

def main():
  hello


## Prompt Engineering

In [12]:
#text = "Write me a poem about Machine Learning"
text = "머신러닝에 대한 시를 써주세요."

In [13]:
prompt = f"""
Waktaverse-Gemma, a Korean language model, capable of performing various natural language processing tasks such as text generation, question answering, translation, summarization, and more. 

Please respond to the following text delimited by triple backticks in Korean.
'''{text}'''
Use Korean only.
"""

In [14]:
response = generate_response(prompt)
print(response)

user
Waktaverse-Gemma, a Korean language model, capable of performing various natural language processing tasks such as text generation, question answering, translation, summarization, and more. 

Please respond to the following text delimited by triple backticks in Korean.
'''머신러닝에 대한 시를 써주세요.'''
Use Korean only.
model
머신 러닝, 컴퓨터 과학의 한 분야
데이터를 학습하여 텍스트, 이미지, 음악, 영상 등 다양한 종류의 정보를 분석하고 생성하는 능력을 가집니다.
지식을 축적하는 과정에서 머신 러닝은 인공지식의 한 부분으로 자 자유로운 학습 과정을 거쳐 컴퓨터 시스템의 모든 부분에서 인간 수준의 성능을 보이거나 능가하는 수준으로 발달합니다.
머신 러닝은 인공지식의 한 부분으로, 인간 수준의 성능을 가진 컴퓨터 시스템을 만듭니다. 머신 러닝은 크게 두 가지 주요 분야로 구성됩니다. 첫 번째 분야는 학습된 모델을 사용하여 새로운 데이터를 예측하는 것입니다. 두 번째 분야는 학습된 모델을 사용하여 새로운 데이터를 예측하는 것입니다. 두 번째 분야는 학습된 모델을 사용하여 새로운 데이터를 예측하는 것입니다.
머신 러닝은 인공지식 기술의 한 부분으로, 인간 수준의 성능을 가진 컴퓨터 시스템을 만듭니다. 머신 러닝은 크게 두 가지 주요 분야로 구성됩니다. 첫 번째 분야는 학습된 모델을 사용하여 새로운 데이터를 예측하는 것입니다. 두 번째 분야는 학습된 모델을 사용하여 새로운 데이터를 예측하는 것입니다. 두 번째 분야는 학습된 모델을 사용하여 새로운 데이터를 예측하는 것입니다.
